##### Copyright 2023 The MediaPipe Authors. All Rights Reserved.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Audio Classification with MediaPipe Tasks

In this notebook you will use the MediaPipe Tasks API to classify audio.

## Preparation
The first thing you will need to do is install the necessary dependencies for this sample.

In [ ]:
!pip install -q sounddevice==0.4.4
!pip install -q mediapipe==0.10.0


The next step you will take is downloading an off-the-shelf model for audio classification. In this case you will use the YAMNet model, which is designed to classify audio in 0.975 second segments, though you are also able to use others, including your own custom models, with MediaPipe Tasks.

In [ ]:
!wget -O classifier.tflite -q https://storage.googleapis.com/mediapipe-models/audio_classifier/yamnet/float32/1/yamnet.tflite

## Performing Audio Classification
Now that you have the necessary dependencies, it's time to start classifying some audio! While there are a variety of ways to retrieve audio clips, this example will download a `.wav` file of someone speaking.

In [ ]:
import urllib

audio_file_name = 'speech_16000_hz_mono.wav'
url = f'https://storage.googleapis.com/mediapipe-assets/{audio_file_name}'
urllib.request.urlretrieve(url, audio_file_name)

You can go ahead and test that your file downloaded correctly by displaying a playback widget.

In [ ]:
from IPython.display import Audio, display

file_name = 'speech_16000_hz_mono.wav'
display(Audio(file_name, autoplay=False))

Once everything looks good, you can start performing inference. You will start by creating the options that are necessary for associating your model with the Audio Classifier, as well as some other customizations.

Next, you will create your Classifier and read some information from your downloaded audio file, as well as segment the clip into smaller (0.975 seconds, in this case) clips before classifying them.

Finally, you will loop through the audio file in increments of 975 (the amount of seconds per clip in millesconds) to display the classification results.

In [ ]:
import numpy as np

from mediapipe.tasks import python
from mediapipe.tasks.python.components import containers
from mediapipe.tasks.python import audio
from scipy.io import wavfile

# Customize and associate model for Classifier
base_options = python.BaseOptions(model_asset_path='classifier.tflite')
options = audio.AudioClassifierOptions(
    base_options=base_options, max_results=4)

# Create classifier, segment audio clips, and classify
with audio.AudioClassifier.create_from_options(options) as classifier:
  sample_rate, wav_data = wavfile.read(audio_file_name)
  audio_clip = containers.AudioData.create_from_array(
      wav_data.astype(float) / np.iinfo(np.int16).max, sample_rate)
  classification_result_list = classifier.classify(audio_clip)

  assert(len(classification_result_list) == 5)

# Iterate through clips to display classifications
  for idx, timestamp in enumerate([0, 975, 1950, 2925]):
    classification_result = classification_result_list[idx]
    top_category = classification_result.classifications[0].categories[0]
    print(f'Timestamp {timestamp}: {top_category.category_name} ({top_category.score:.2f})')